In [1]:
# 현재까지 완성버전

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
import pandas as pd

current_time = datetime.now().strftime('%y-%m-%d')

def setup_driver():
    """Chrome 드라이버를 설정하고 반환합니다."""
    options = Options()
    options.add_experimental_option("detach", True)
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def switch_frame(driver, frame):
    """프레임을 변경합니다."""
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

def time_wait(driver, num, code):
    """요소를 찾을 때까지 대기하는 함수"""
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except Exception as e:
        print(f"{code} 태그를 찾지 못하였습니다. 오류: {e}")
        driver.quit()
    return wait

def search_places(driver, location):
    """주어진 위치에서 검색합니다."""
    # 네이버 지도 화면의 URL
    url = "https://map.naver.com/v5/search"
    driver.get(url)

    if location in ['동대문 관광특구', '남산공원']:
        # 검색창 찾기 및 검색어 입력
        time_wait(driver, 10, 'div.input_box > input.input_search')
        
        search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
        search.send_keys(location)  # 위치 검색어 입력
        search.send_keys(Keys.ENTER)  # 엔터 버튼 누르기
        time.sleep(5)
    else:
        # 검색창 찾기 및 검색어 입력
        time_wait(driver, 10, 'div.input_box > input.input_search')
        
        search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
        search.send_keys(f'서울+{location}')  # 위치 검색어 입력
        search.send_keys(Keys.ENTER)  # 엔터 버튼 누르기
        time.sleep(5)

    # 검색창을 찾고 기존 텍스트 지우기
    search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
    search.send_keys(Keys.CONTROL, 'a')  # 모든 텍스트 선택
    search.send_keys(Keys.BACKSPACE)  # 백스페이스로 텍스트 삭제
    time.sleep(1)  # 잠시 대기
    
    # 새로운 검색어 입력
    # search.send_keys('음식점')  # 음식점 검색어 입력
    # search.send_keys('숙박시설')  # 숙박시설 검색어 입력
    search.send_keys('카페')  # 카페 검색어 입력
    
    search.send_keys(Keys.ENTER)  # 엔터 버튼 누르기
    time.sleep(3)

    # 프레임 변경
    switch_frame(driver, driver.find_element(By.CSS_SELECTOR, 'iframe#searchIframe'))

    # 검색 결과가 로드될 때까지 대기
    # 음식점, 카페
    time_wait(driver, 10, ".place_bluelink.TYaxT")
    # 숙박시설
    # time_wait(driver, 15, "span.YwYLL")

def extract_place_names(driver):
    """리스트를 추출합니다."""
    place_names = set()
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 리스트 추출
        # 음식점, 카페
        places = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT")
        # 숙박시설
        # places = driver.find_elements(By.CSS_SELECTOR, "span.YwYLL")

        for place in places:
            place_names.add(place.text)

        # 스크롤을 끝까지 내린다.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # 페이지 로딩 대기

        # 새로 스크롤된 높이를 가져와 이전과 비교
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 스크롤이 더 이상 내려가지 않으면 종료
        last_height = new_height

    return place_names

def get_place_names_for_locations(locations):
    """여러 지역에 대해 리스트를 추출하고 DataFrame으로 반환합니다."""
    driver = setup_driver()
    all_data = []

    for location in locations:
        print(f"'{location}'에서 검색 중...")
        search_places(driver, location)
        place_names = extract_place_names(driver)

        # 결과를 데이터에 추가
        for place_name in place_names:
            all_data.append({'검색어': location, '검색결과': place_name})

        print(f"'{location}'의 목록:")
        for name in place_names:
            print(name)

    driver.quit()

    # DataFrame으로 변환
    df = pd.DataFrame(all_data)
    
    # 파일로 저장
    # df.to_excel(f'../data/음식점_검색결과_{current_time}.xlsx', index=False)
    # print("데이터가 '음식점_검색결과.xlsx' 파일에 저장되었습니다.")
    
    # df.to_excel(f'../data/숙박시설_검색결과_{current_time}.xlsx', index=False)
    # print("데이터가 '숙박시설_검색결과.xlsx' 파일에 저장되었습니다.")
    
    df.to_excel(f'../data/카페_검색결과_{current_time}.xlsx', index=False)
    print("데이터가 '카페_검색결과.xlsx' 파일에 저장되었습니다.")

    return df

# 예제 사용
locations = [
    # 발달상권
    "성수카페거리",
    "가로수길",
    "북촌한옥마을",
    "인사동·익선동",
    "광장(전통)시장",
    "압구정로데오거리",
    
    # 인구밀집지역
    "홍대입구역(2호선)",
    "이태원역",
    "강남역",
    "서울식물원·마곡나루역",
    
    # 관광특구
    "동대문 관광특구",
    "명동 관광특구",
    "잠실 관광특구",
    "종로·청계 관광특구",
    
    # 고궁 문화유산
    "경복궁",
    "광화문·덕수궁",
    "창덕궁·종묘",
    "보신각",
    
    # 공원
    "반포한강공원",
    "고척돔",
    "남산공원",
    "잠실종합운동장",
    "서울숲공원",
    "국립중앙박물관·용산가족공원",
    "서울대공원"
]

place_names_df = get_place_names_for_locations(locations)

'성수카페거리'에서 검색 중...
'성수카페거리'의 목록:
아도르 성수
아쿠아산타 성수카페
LOOOP 루프 베이커리 카페
네모네
쎈느Scene
무브모브 성수
따우전드 성수점
텅 성수 스페이스
베통 성수 플래그십
비아트 성수
'가로수길'에서 검색 중...
'가로수길'의 목록:
5to7
LOOOP 루프 베이커리 카페
무브먼트
듀자미
구욱희씨 서울숲 본점
빈브라더스 파미에스테이션
mtl 한남
네모네
윌비스콘
따우전드 신사점
'북촌한옥마을'에서 검색 중...
'북촌한옥마을'의 목록:
땡스오트 안국
청수당 베이커리
사월의물고기
슬로운
카페창덕궁
커피사는세상
낙원장
목음당
익선주택
서울커피 익선본점
'인사동·익선동'에서 검색 중...


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=128.0.6613.119)
Stacktrace:
	GetHandleVerifier [0x00A38213+26163]
	(No symbol) [0x009C9CC4]
	(No symbol) [0x008C24C3]
	(No symbol) [0x0089E27B]
	(No symbol) [0x0093192F]
	(No symbol) [0x00943F99]
	(No symbol) [0x0092AA56]
	(No symbol) [0x008FBE89]
	(No symbol) [0x008FC8CD]
	GetHandleVerifier [0x00D0D313+2996019]
	GetHandleVerifier [0x00D61B89+3342249]
	GetHandleVerifier [0x00AC7AEF+614159]
	GetHandleVerifier [0x00ACF17C+644508]
	(No symbol) [0x009D27FD]
	(No symbol) [0x009CF6F8]
	(No symbol) [0x009CF895]
	(No symbol) [0x009C1C16]
	BaseThreadInitThunk [0x7521FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x770180CE+286]
	RtlGetAppContainerNamedObjectPath [0x7701809E+238]
